[Transformer 모델 설계]
우리는 "Encoder-Only Transformer" (BERT 스타일) 구조를 사용할 것입니다.
입력 (Input Sequence):
모양: (Batch, Sequence_Length, Feature_Dim)
Sequence_Length: 과거 10개의 이벤트 (LSTM보다 더 길게 봐도 됨)
Feature_Dim: 스텝마다 [공 위치, 골대 정보, 액션ID, + 주변 선수들 정보]
모델 구조:
Embedding Layer: 좌표와 액션ID 등을 고차원 벡터로 변환
Positional Encoding: 순서 정보 주입
Transformer Encoder: Self-Attention을 통해 "아, 3스텝 전의 패스가 중요했구나"를 파악
Output Head: 마지막 토큰의 정보를 받아서 (next_x, next_y) 예측
[작업 순서]
전처리: pre_test의 시퀀스 생성 로직을 가져오되, 주변 선수 정보도 포함시킵니다.
모델링: PyTorch로 간단한 Transformer 클래스를 만듭니다.
학습: GPU 서버(또는 로컬)에서 학습합니다. Transformer는 병렬 처리가 잘 돼서 LSTM보다 학습도 빠릅니다.
바로 Transformer 모델 코드를 짜드릴까요?

이 코드는 다음 단계로 구성됩니다:
데이터 전처리: 전체 데이터를 로드하고, 각 에피소드별로 시퀀스 데이터(Sequence Data)를 만듭니다. 피처에는 [x, y, 골대거리, 골대각도, 액션ID]를 포함합니다. (주변 선수 정보는 일단 데이터가 무거워질 수 있으니, 1차적으로 가벼운 버전으로 먼저 성능을 봅니다.)
Transformer 모델 정의: PyTorch의 nn.TransformerEncoder를 사용하여 시계열 패턴을 학습하는 모델을 만듭니다.
학습 및 추론: 모델을 학습시키고, test.csv에 대한 제출 파일을 생성합니다.

## [현재 모델링 진행 상황 요약]

### 1. 데이터 엔지니어링
- **피처 확장 (8차원)**: 기존 [x, y, 거리, 각도, action]에 추가로:
  - **속도(Velocity)**: [vx, vy] (이전 스텝과의 좌표 차이)
  - **결과(Result)**: Successful(1), Unsuccessful(-1), Others(0) (성공/실패 여부를 힌트로 제공)
- **데이터 증강 (Data Augmentation)**: 학습 데이터 4배 확대
  - 원본 + 상하반전 + 좌우반전 + 상하좌우반전
  - **순서 보장**: 시계열 흐름이 끊기지 않도록 [원본전체] -> [증강전체] 순으로 배치

### 2. 모델 구조 (Transformer Encoder)
- **입력**: (Batch, Seq_Len=10, Feature=8)
- **Hyperparams**: d_model=256, nhead=8, layers=3, dropout=0.2
- **출력**: 마지막 시점의 (x, y) 좌표 예측


네, 죄송합니다. 마크다운 내용을 아래에 제공해 드리겠습니다. 복사해서 **3번 셀(제목 있는 곳)**에 붙여넣으시면 됩니다.

---

### [코드 실행 순서 및 개요]

이 노트북은 축구 경기 데이터를 사용하여 **Transformer 기반의 다음 좌표 예측 모델**을 학습하고 추론하는 과정을 담고 있습니다.

### 1. 환경 설정 및 데이터 로드
- **라이브러리**: PyTorch, Pandas, Numpy 등 필수 패키지 임포트
- **데이터 로드**: `train.csv`를 읽어옵니다.
- **정규화**: 좌표(105x68)와 Action ID를 0~1 범위로 스케일링합니다.
- **결과 매핑**: `result_name`을 수치화합니다 (성공=1, 실패=-1, 기타=0).
- **데이터 분할**: **Data Leakage 방지**를 위해 `game_episode` 단위로 학습/검증 데이터를 나눕니다.

### 2. 데이터셋 구축 (Feature Engineering & Augmentation)
- **시퀀스 생성**: 각 에피소드에서 과거 **10개**의 이벤트를 묶어 하나의 입력 시퀀스로 만듭니다.
- **피처 확장 (8차원)**:
  1. `x`, `y` (좌표)
  2. `distance` (골대까지 거리)
  3. `angle` (골대까지 각도)
  4. `action_id` (행동 종류)
  5. `vx`, `vy` (**속도**: 이전 스텝과의 차이)
  6. `result` (**결과**: 성공/실패 여부)
- **데이터 증강 (학습셋만 적용)**: 데이터 양을 **4배**로 늘립니다.
  1. 원본 데이터
  2. 상하 반전 (y축 대칭)
  3. 좌우 반전 (x축 대칭)
  4. 상하좌우 반전 (점 대칭)

### 3. 모델 정의 (SoccerTransformer)
- **구조**: Encoder-Only Transformer (BERT 스타일)
- **Embedding**: 8차원 피처 -> 256차원 벡터
- **Positional Encoding**: 시계열 순서 정보 주입
- **Encoder Layers**: 3층의 Self-Attention 레이어 (Head=8)
- **Output Head**: 마지막 시점의 벡터를 받아 (x, y) 좌표 예측

### 4. 학습 (Training Loop)
- **Optimizer**: Adam (lr=0.0005)
- **Scheduler**: ReduceLROnPlateau (검증 성능 정체 시 학습률 감소)
- **Process**: 100 Epoch 동안 학습하며, 매 Epoch마다 검증셋으로 성능(거리 오차)을 평가합니다.

### 5. 추론 및 제출 (Inference)
- `test.csv`의 경로를 통해 테스트 데이터를 로드합니다.
- 학습과 동일한 전처리(정규화, 피처 생성)를 거칩니다.
- 학습된 모델로 다음 좌표를 예측하고, 원래 크기(105x68)로 복원하여 제출 파일(`submission.csv`)을 생성합니다.

In [1]:
# ==========================================================
# [축구 경기 예측을 위한 Transformer 모델링 스크립트]
# - 작업: 데이터 로드, 전처리(피처 엔지니어링, 증강), 모델 정의, 학습, 추론
# - 특징: 8차원 피처(좌표, 속도, 결과 등), 4배 데이터 증강, Transformer Encoder 구조
# ==========================================================

import os
import math
from time import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm

CONFIG = {
    "field_dims": (105.0, 68.0),
    "action_scale": 60.0,
    "goal_xy": (1.0, 0.5),
    "seq_len": 10,
    "feature_size": 8,
    "augmentations": {
        "vertical": True,
        "horizontal": True,
        "both": True,
    },
    "model": {
        "d_model": 256,
        "nhead": 8,
        "num_layers": 3,
        "dropout": 0.2,
        "lr": 5e-4,
    },
    "training": {
        "epochs": 100,
        "batch_size": 64,
        "log_interval": 5,
    },
    "fallback_xy": (52.5, 34.0),
}


def map_result(value):
    """Map textual result labels to numeric hints."""
    if value == 'Successful':
        return 1.0
    if value == 'Unsuccessful':
        return -1.0
    return 0.0


def preprocess_dataframe(df, config, is_train=True):
    """Normalize coordinates/action id and attach numeric result column."""
    field_x, field_y = config["field_dims"]
    df = df.copy()
    for col in ("start_x", "end_x"):
        if col in df.columns:
            df[col] = df[col].astype(float) / field_x
    for col in ("start_y", "end_y"):
        if col in df.columns:
            df[col] = df[col].astype(float) / field_y
    if "action_id" in df.columns:
        df["action_id"] = df["action_id"].fillna(0) / config["action_scale"]
    else:
        df["action_id"] = 0.0
    if "result_name" in df.columns:
        df["result_mapped"] = df["result_name"].apply(map_result)
    elif "result_mapped" not in df.columns:
        df["result_mapped"] = 0.0
    else:
        df["result_mapped"] = df["result_mapped"].fillna(0)
    return df


def build_feature_matrix(coords, actions, results, goal_xy):
    """Compose the 8D feature matrix for a single episode."""
    goal_x, goal_y = goal_xy
    dist = np.sqrt((coords[:, 0] - goal_x) ** 2 + (coords[:, 1] - goal_y) ** 2).reshape(-1, 1)
    angle = np.arctan2(coords[:, 1] - goal_y, coords[:, 0] - goal_x).reshape(-1, 1)
    velocities = np.zeros_like(coords)
    if len(coords) > 1:
        velocities[1:, 0] = coords[1:, 0] - coords[:-1, 0]
        velocities[1:, 1] = coords[1:, 1] - coords[:-1, 1]
    return np.hstack([coords, dist, angle, actions, velocities, results])


def pad_or_truncate(feats, seq_len, feature_size):
    """Match sequence length by trimming old steps or front-padding zeros."""
    if len(feats) >= seq_len:
        return feats[-seq_len:]
    padded = np.zeros((seq_len, feature_size))
    padded[-len(feats):] = feats
    return padded


def generate_variants(coords, target, config, enable_augment):
    """Create augmented coordinate/target pairs according to config."""
    variants = [(coords, target)]
    if not enable_augment:
        return variants
    aug_cfg = config["augmentations"]
    if aug_cfg.get("vertical"):
        coords_v = coords.copy()
        coords_v[:, 1] = 1.0 - coords_v[:, 1]
        target_v = target.copy()
        target_v[1] = 1.0 - target_v[1]
        variants.append((coords_v, target_v))
    if aug_cfg.get("horizontal"):
        coords_h = coords.copy()
        coords_h[:, 0] = 1.0 - coords_h[:, 0]
        target_h = target.copy()
        target_h[0] = 1.0 - target_h[0]
        variants.append((coords_h, target_h))
    if aug_cfg.get("both"):
        coords_hv = coords.copy()
        coords_hv[:, 0] = 1.0 - coords_hv[:, 0]
        coords_hv[:, 1] = 1.0 - coords_hv[:, 1]
        target_hv = target.copy()
        target_hv[0] = 1.0 - target_hv[0]
        target_hv[1] = 1.0 - target_hv[1]
        variants.append((coords_hv, target_hv))
    return variants


def create_sequences(df, config, augment=False):
    """Convert an entire dataframe into stacked sequences and targets."""
    sequences, targets = [], []
    seq_len = config["seq_len"]
    feature_size = config["feature_size"]
    grouped = df.groupby('game_episode')
    for _, group in tqdm(grouped, desc=f"시퀀스 생성(Augment={augment})"):
        group = group.sort_values('time_seconds')
        coords = group[['start_x', 'start_y']].values
        actions = group['action_id'].values.reshape(-1, 1)
        results = group['result_mapped'].values.reshape(-1, 1)
        target = group[['end_x', 'end_y']].values[-1].copy()
        for coords_variant, target_variant in generate_variants(coords, target, config, augment):
            feats = build_feature_matrix(coords_variant, actions, results, config['goal_xy'])
            seq = pad_or_truncate(feats, seq_len, feature_size)
            sequences.append(seq)
            targets.append(target_variant)
    return np.array(sequences), np.array(targets)


def load_match_sequence(file_path, config):
    """Load and preprocess a single match csv into a padded sequence."""
    if not os.path.exists(file_path):
        return None
    temp_df = pd.read_csv(file_path)
    temp_df = preprocess_dataframe(temp_df, config, is_train=False)
    coords = temp_df[['start_x', 'start_y']].values
    actions = temp_df['action_id'].values.reshape(-1, 1)
    results = temp_df['result_mapped'].values.reshape(-1, 1)
    feats = build_feature_matrix(coords, actions, results, config['goal_xy'])
    return pad_or_truncate(feats, config['seq_len'], config['feature_size'])


def run_inference(model, meta_df, base_path, config, device):
    """Iterate over metadata rows and generate predictions."""
    model.eval()
    preds_x, preds_y = [], []
    fallback_x, fallback_y = config['fallback_xy']
    with torch.no_grad():
        for _, row in tqdm(meta_df.iterrows(), total=len(meta_df)):
            file_path = os.path.join(base_path, row['path'][2:])
            seq = load_match_sequence(file_path, config)
            if seq is None:
                preds_x.append(fallback_x)
                preds_y.append(fallback_y)
                continue
            input_tensor = torch.FloatTensor(seq).unsqueeze(0).to(device)
            pred = model(input_tensor).cpu().numpy()[0]
            preds_x.append(pred[0] * config['field_dims'][0])
            preds_y.append(pred[1] * config['field_dims'][1])
    return preds_x, preds_y


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        length = x.size(1)
        return x + self.pe[:, :length, :]


class SoccerTransformer(nn.Module):
    """Encoder-only Transformer tailored for soccer sequence regression."""

    def __init__(self, config):
        super().__init__()
        model_cfg = config['model']
        feature_size = config['feature_size']
        self.embedding = nn.Linear(feature_size, model_cfg['d_model'])
        self.pos_encoder = PositionalEncoding(model_cfg['d_model'])
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_cfg['d_model'],
            nhead=model_cfg['nhead'],
            dim_feedforward=512,
            dropout=model_cfg['dropout'],
            batch_first=True,
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=model_cfg['num_layers'])
        self.fc_out = nn.Linear(model_cfg['d_model'], 2)

    def forward(self, src):
        x = self.embedding(src)
        x = self.pos_encoder(x)
        encoded = self.transformer_encoder(x)
        return self.fc_out(encoded[:, -1, :])


def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)


def evaluate(model, val_inputs, val_targets, criterion, device, field_dims):
    model.eval()
    with torch.no_grad():
        outputs = model(val_inputs)
        loss = criterion(outputs, val_targets).item()
        diff_x = (outputs[:, 0] - val_targets[:, 0]) * field_dims[0]
        diff_y = (outputs[:, 1] - val_targets[:, 1]) * field_dims[1]
        val_score = torch.mean(torch.sqrt(diff_x ** 2 + diff_y ** 2)).item()
    return loss, val_score


print("데이터 로드 중...")
train_df = pd.read_csv("../open_track1/train.csv")
train_df = preprocess_dataframe(train_df, CONFIG, is_train=True)
unique_episodes = train_df['game_episode'].unique()
train_episodes, val_episodes = train_test_split(unique_episodes, test_size=0.2, random_state=42)
train_df_split = train_df[train_df['game_episode'].isin(train_episodes)].copy()
val_df_split = train_df[train_df['game_episode'].isin(val_episodes)].copy()
print(f"학습 에피소드: {len(train_episodes)}개, 검증 에피소드: {len(val_episodes)}개")

print("학습 데이터셋 생성 중 (4배 증강 적용)...")
X_train, y_train = create_sequences(train_df_split, CONFIG, augment=True)
print(f"학습 데이터 Shape: {X_train.shape}")
print("검증 데이터셋 생성 중 (증강 미적용)...")
X_val, y_val = create_sequences(val_df_split, CONFIG, augment=False)
print(f"검증 데이터 Shape: {X_val.shape}")

train_loader = DataLoader(
    TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train)),
    batch_size=CONFIG['training']['batch_size'],
    shuffle=True,
)
X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")
model = SoccerTransformer(CONFIG).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=CONFIG['model']['lr'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
print(model)

print("[Transformer 학습 시작]")
training_cfg = CONFIG['training']
X_val_tensor = X_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)
for epoch in range(training_cfg['epochs']):
    start = time()
    avg_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_score = evaluate(model, X_val_tensor, y_val_tensor, criterion, device, CONFIG['field_dims'])
    scheduler.step(val_loss)
    if (epoch + 1) % training_cfg['log_interval'] == 0:
        current_lr = optimizer.param_groups[0]['lr']
        elapsed = time() - start
        print(f"Epoch {epoch+1:02d} | Loss: {avg_loss:.5f} | Val Score: {val_score:.4f} | LR: {current_lr:.2e} | Time: {elapsed:.1f}s")

print("[추론 시작]")
test_meta = pd.read_csv("../open_track1/test.csv")
preds_x, preds_y = run_inference(model, test_meta, "../open_track1", CONFIG, device)

submission = pd.read_csv("../open_track1/sample_submission.csv")
submission['end_x'] = preds_x
submission['end_y'] = preds_y
output_name = "submission_transformer_v5_result_feat1.csv"
submission.to_csv(output_name, index=False)
print(f"저장 완료: {output_name}")



데이터 로드 중...
학습 에피소드: 12348개, 검증 에피소드: 3087개
학습 데이터셋 생성 중 (4배 증강 적용)...


시퀀스 생성(Augment=True): 100%|██████████| 12348/12348 [00:09<00:00, 1240.31it/s]


학습 데이터 Shape: (49392, 10, 8)
검증 데이터셋 생성 중 (증강 미적용)...


시퀀스 생성(Augment=False): 100%|██████████| 3087/3087 [00:02<00:00, 1524.01it/s]


검증 데이터 Shape: (3087, 10, 8)
Using Device: cuda
SoccerTransformer(
  (embedding): Linear(in_features=8, out_features=256, bias=True)
  (pos_encoder): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=512, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (fc_out): Linear(in_features=256, out_features=2, bias=True)
)
[Transformer 학습 시작]
Epoch 05 | Loss: 0.04096 | V

100%|██████████| 2414/2414 [00:11<00:00, 214.76it/s]

저장 완료: submission_transformer_v5_result_feat1.csv
